## DIA 102: Proteger tu API REST con Autenticación JWT (JSON Web Token)

Ahora que tienes una API REST funcional, es hora de asegurarla para que solo usuarios autorizados puedan usarla.
Hoy aprenderás a proteger tu endpoint usando tokens JWT, una forma moderna y ligera de autenticación.

✅ ¿Qué aprenderás hoy?

🔐 Qué es un JWT y cómo funciona.

🧾 Cómo generar tokens al iniciar sesión.

🛡️ Cómo verificar el token en rutas protegidas.

🧩 1. Instala la librería de JWT para Flask
bash
Copiar
Editar
pip install flask-jwt-extended
🧩 2. Configuración inicial en app.py
python
Copiar
Editar
from flask_jwt_extended import (
    JWTManager, create_access_token,
    jwt_required, get_jwt_identity
)

app.config['JWT_SECRET_KEY'] = 'clave_secreta_segura'  # 🔐 cámbiala en producción
jwt = JWTManager(app)
🧩 3. Ruta para iniciar sesión y obtener el token
python
Copiar
Editar
@app.route('/api/login', methods=['POST'])
def api_login():
    data = request.get_json()
    email = data.get('email')
    password = data.get('password')

    user = User.query.filter_by(email=email).first()

    if user and check_password_hash(user.password, password):
        access_token = create_access_token(identity=user.id)
        return jsonify(access_token=access_token), 200
    else:
        return jsonify({"msg": "Credenciales inválidas"}), 401
🧩 4. Proteger la ruta de predicción
python
Copiar
Editar
@app.route('/api/prediccion', methods=['POST'])
@jwt_required()
def api_prediccion():
    data = request.get_json()
    required_fields = ['f1', 'f2', 'f3', 'f4']

    if not all(field in data for field in required_fields):
        return jsonify({'error': 'Faltan campos f1, f2, f3, f4'}), 400

    try:
        entrada = [[
            float(data['f1']),
            float(data['f2']),
            float(data['f3']),
            float(data['f4'])
        ]]
        pred = modelo.predict(entrada)[0]
        return jsonify({'prediccion': int(pred)})

    except Exception as e:
        return jsonify({'error': str(e)}), 500
📥 5. Cómo usarlo desde Postman o curl
🧪 Paso 1: Obtener token
bash
Copiar
Editar
curl -X POST http://127.0.0.1:5000/api/login \
     -H "Content-Type: application/json" \
     -d '{"email":"usuario@email.com","password":"tu_clave"}'
Respuesta:

json
Copiar
Editar
{
  "access_token": "eyJ0eXAiOiJK..."
}
🧪 Paso 2: Enviar predicción usando token
bash
Copiar
Editar
curl -X POST http://127.0.0.1:5000/api/prediccion \
     -H "Content-Type: application/json" \
     -H "Authorization: Bearer TU_TOKEN_AQUI" \
     -d '{"f1":1.0,"f2":2.0,"f3":3.0,"f4":4.0}'
